In [17]:
import os

os.environ['KALDI_ROOT'] = '/home/faber6911/kaldi/'

import kaldi_io
import numpy as np
import librosa
import IPython.display as ipd
import random
import kaldiio
from kaldiio import WriteHelper
from utils import add_noise
import jit

ModuleNotFoundError: No module named 'jit'

In [2]:
ark_path_train = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/train/noisy_audio.ark'
scp_path_train = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/train/noisy_audio.scp'
ark_path_test = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/test/noisy_audio.ark'
scp_path_test = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/test/noisy_audio.scp'
train_augmentation = 10
test_augmentation = 1

In [14]:
for count, line in enumerate(open('../data/test/wav.scp')):
#     utt, path = line.rstrip().split()
#     clean_audio, _ = librosa.load(path, sr = 16000)
#     ipd.Audio(clean_audio, rate = 16000)
#     break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


In [6]:
ipd.Audio(clean_audio, rate = 16000)

In [10]:
_, noise_path = open('../data/musan_{}.scp'.format('speech')).readlines()[10].rstrip().split()

In [11]:
noise_audio, _ = librosa.load(noise_path, sr = 16000)
noisy_audio = add_noise(clean_audio, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))

In [12]:
ipd.Audio(noisy_audio, rate = 16000)

In [8]:
print('Adding noise to train audio and augmenting each file {} times'.format(train_augmentation*3))
writer = WriteHelper('ark,scp:{},{}'.format(ark_path_train, scp_path_train), compression_method=2)

noise_choice = {'music':659, 'noise':929, 'speech':425}

for count, line in enumerate(open('../data/train/wav.scp')):
    # clean audio path
    utt, path = line.rstrip().split()
    # clean audio file
    clean_audio, _ = librosa.load(path, sr = 16000)
    # now for every noise type we augment n times the clean audio file using random noise audio files
    for noise_type in noise_choice:
        for aug in range(train_augmentation):
            noise_track = np.random.randint(0, noise_choice[noise_type])
            _, noise_path = open('../data/musan_{}.scp'.format(noise_type)).readlines()[noise_track].rstrip().split()
            noise_audio, _ = librosa.load(noise_path, sr = 16000)
            noisy_audio = add_noise(clean_audio, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))
            # write ark and associated scp file in train directory
            writer(path, noisy_audio.reshape(-1, 1))
    if count % 100 == 0:
        print('Augmented {} audio files'.format(count))
    
writer.close()
print('Augmented train data')

Adding noise to train audio and augmenting each file 30 times
Augmented 0 audio files
Augmented 100 audio files
Augmented 200 audio files
Augmented 300 audio files
Augmented 400 audio files
Augmented 500 audio files
Augmented 600 audio files
Augmented 700 audio files
Augmented 800 audio files
Augmented 900 audio files
Augmented 1000 audio files
Augmented 1100 audio files
Augmented 1200 audio files
Augmented 1300 audio files
Augmented 1400 audio files
Augmented 1500 audio files
Augmented 1600 audio files
Augmented 1700 audio files
Augmented 1800 audio files
Augmented 1900 audio files
Augmented 2000 audio files
Augmented 2100 audio files
Augmented 2200 audio files
Augmented 2300 audio files
Augmented 2400 audio files
Augmented train data


In [10]:
print('Adding noise to test audio and augmenting each file {} times'.format(test_augmentation*3))
writer = WriteHelper('ark,scp:{},{}'.format(ark_path_test, scp_path_test), compression_method=2)

noise_choice = {'music':659, 'noise':929, 'speech':425}

for count, line in enumerate(open('../data/test/wav.scp')):
    # clean audio path
    utt, path = line.rstrip().split()
    # clean audio file
    clean_audio, _ = librosa.load(path, sr = 16000)
    # now for every noise type we augment n times the clean audio file using random noise audio files
    for noise_type in noise_choice:
        for aug in range(test_augmentation):
            noise_track = np.random.randint(0, noise_choice[noise_type])
            _, noise_path = open('../data/musan_{}.scp'.format(noise_type)).readlines()[noise_track].rstrip().split()
            noise_audio, _ = librosa.load(noise_path, sr = 16000)
            noisy_audio = add_noise(clean_audio, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))
            # write ark and associated scp file in train directory
            writer(path, noisy_audio.reshape(-1, 1))
    if count % 100 == 0:
        print('Augmented {} audio files'.format(count))
    
writer.close()
print('Augmented test data')

Adding noise to test audio and augmenting each file 3 times
Augmented 0 audio files
Augmented 100 audio files
Augmented 200 audio files
Augmented test data
